In [ ]:
import json
from pathlib import Path
import datetime
import shutil
import os

import pandas as pd

Use this notebook to parse server commands, store them locally and plot profiling info.

These commands can then be used for replaying, which can be done for benchmarking and testing purposes, and is generally faster than waiting for data to be sent from the server.

In [ ]:
# Set your data directory here. This is the only cell you need to modify
DATA_DIR = Path('/home/gajop/Beyond All Reason')

In [ ]:
INFOLOG = DATA_DIR / "infolog.txt"
COMMANDS_LOG = DATA_DIR / "commands.log"
LOG_CAPTURES_DIR = DATA_DIR / "log_captures"
COMMANDS_JSON = DATA_DIR / "commands.json"

In [ ]:
# Utility functions
def parse_commands(path: Path) -> list[dict]:
    with open(path, "r") as f:
        commands = f.readlines()
    commands = [ json.loads(cmd) for cmd in commands ]
    if len(commands) == 0:
        raise ValueError("No commands found. Did you forget to enable command capture in dbg_command_capture.lua or login to the server?")
    return commands

def make_dataframe_from_commands(commands: list[dict]) -> pd.DataFrame:
    df = pd.DataFrame(commands)
    df = df.rename(columns = {'arg1': 'command'})
    df['command_type'] = df['command'].apply(parse_command_type)
    df['duration'] = df['end_time'] - df['start_time']
    return df

def parse_command_type(command: str) -> str:
    if command.startswith('#'):
        command = command.split('#', maxsplit=1)[1].split(maxsplit=1)[1]

    return command.split()[0]

def save_commands_for_replay(df: pd.DataFrame, path: Path) -> None:
    commands = df['command'].values
    json.dump(list(commands), open(path, "w"))

In [ ]:
def plot_entry(path: Path) -> None:
    df = pd.read_csv(path)

    print(f'Total duration: {df["duration"].sum()}')

    # Render time info
    groupby = df.groupby(['command_type'])[['duration', 'function']]

    ax = groupby.mean(numeric_only=True).plot.barh(title='Average command time[s]')
    ax.figure.savefig("new_avg.png")
    ax = groupby.sum(numeric_only=True).plot.barh(title='Aggregate command time[s]')
    ax.figure.savefig("new_agg.png")

    df_stats = groupby.sum(numeric_only=True)
    df_stats = df_stats.rename(columns={'duration': 'total_time(ms)'})
    df_stats['total_time(ms)'] = (df_stats['total_time(ms)'] * 1000.0).round()
    count = groupby.count()['function']
    df_stats['mean_time(ms)'] = df_stats['total_time(ms)'] / count
    df_stats['count'] = count

    df_stats = df_stats.sort_values(by='total_time(ms)', ascending=False)
    display(df_stats)

In [ ]:
# Load, parse, and store captures
os.makedirs(LOG_CAPTURES_DIR, exist_ok=True)

dt = datetime.datetime.fromtimestamp(COMMANDS_LOG.stat().st_ctime)
timestamp = dt.strftime("%Y-%m-%d_%H-%M-%S")

commands = parse_commands(COMMANDS_LOG)
df = make_dataframe_from_commands(commands)
save_commands_for_replay(df, COMMANDS_JSON)

shutil.copy(INFOLOG, LOG_CAPTURES_DIR / f"infolog-{timestamp}.txt")
shutil.copy(COMMANDS_LOG, LOG_CAPTURES_DIR / f"commands-{timestamp}.log")
shutil.copy(COMMANDS_JSON, LOG_CAPTURES_DIR / f"commands-{timestamp}.json")
df.to_csv(LOG_CAPTURES_DIR / f'parsed-{timestamp}.csv', index=False)

display(df)

In [ ]:
plot_entry(LOG_CAPTURES_DIR / f"parsed-{timestamp}.csv")

# Optionally compare with some older entry
# plot_entry(LOG_CAPTURES_DIR / f"parsed-2023-03-21_17-38-09.csv")